## Phase 1: Baseline 
The goal of this phase is to create a baseline model. Note that the word baseline can mean different things. In the course we distinguished three different types of baselines:

1. The simplest possible approach (majority baseline, i.e. everything is positive or noun)
2. A simple machine learning classifier (logistic regression with words as features)
3. The 'state-of-the-art' approach on which you want to improve (your starting point)

### Task: Sentiment classification
The data can be found in the **classification folder** .  
The goal is to **predict the label** in the sentiment field.  
You have to upload the predictions of music_reviews_test_masked.json.gz to CodaLab. (The link will be posted here on monday). Note that the format should match the json files in the repository.  
Also upload a .txt file on LearnIt (one per group) with a short description of your baseline.  

0 - Imports

In [7]:
import numpy as np
import gzip
import json

### 1 - Load the Data

Train

In [8]:
train_data = {}
for value, line in enumerate(gzip.open('../data/classification/music_reviews_train.json.gz')):
    review_data = json.loads(line)
    train_data[value] = {}
    for key, dict_value in review_data.items():
        train_data[value][key] = dict_value

Development

In [9]:
dev_data = {}
for value, line in enumerate(gzip.open('../data/classification/music_reviews_dev.json.gz')):
    review_data = json.loads(line)
    dev_data[value] = {}
    for key, dict_value in review_data.items():
        dev_data[value][key] = dict_value

Test

In [10]:
test_data = {}
for value, line in enumerate(gzip.open('../data/classification/music_reviews_test_masked.json.gz')):
    review_data = json.loads(line)
    test_data[value] = {}
    for key, dict_value in review_data.items():
        test_data[value][key] = dict_value